In [1]:
import os
import sys
from pathlib import Path

# Add project root to path 
project_root = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
sys.path.append(str(project_root))

print(f"Project root: {project_root}")
print(f"Working from: {Path.cwd()}")

Project root: /Users/taishajoseph/Documents/Projects/MDC-Challenge-2025
Working from: /Users/taishajoseph/Documents/Projects/MDC-Challenge-2025/notebooks


In [2]:
from src.dimensionality_reduction import Reducer

2025-08-06 19:32:03,861 - src.helpers - INFO - Logging initialized for /Users/taishajoseph/Documents/Projects/MDC-Challenge-2025/logs/duckdb_utils.log
2025-08-06 19:32:03,861 - src.helpers - INFO - Logging initialized for /Users/taishajoseph/Documents/Projects/MDC-Challenge-2025/logs/duckdb_utils.log
2025-08-06 19:32:07,107 - src.helpers - INFO - Logging initialized for /Users/taishajoseph/Documents/Projects/MDC-Challenge-2025/logs/dimensionality_reduction.log
2025-08-06 19:32:07,107 - src.helpers - INFO - Logging initialized for /Users/taishajoseph/Documents/Projects/MDC-Challenge-2025/logs/dimensionality_reduction.log
2025-08-06 19:32:07,107 - src.helpers - INFO - Logging initialized for /Users/taishajoseph/Documents/Projects/MDC-Challenge-2025/logs/dimensionality_reduction.log


In [3]:
# reduce = Reducer()

In [4]:
# data = reduce.load_datasets_from_duckdb()
# len(data)

In [5]:
# from api.utils.duckdb_utils import DuckDBHelper

# db_helper = DuckDBHelper()

In [6]:
# db_helper.get_database_stats()

In [7]:
# data = db_helper.get_all_datasets()
# len(data)

In [8]:
# Fix: Use the correct database path (go up one directory)
correct_db_path = "../artifacts/mdc_challenge.db"
reduce_fixed = Reducer(db_path=correct_db_path)

print(f"Using database: {correct_db_path}")
data_fixed = reduce_fixed.load_datasets_from_duckdb()
print(f"Found {len(data_fixed)} datasets!")

if len(data_fixed) > 0:
    print(f"First dataset: {data_fixed[0].dataset_id}")


2025-08-06 19:32:07,209 - api.database.duckdb_schema - INFO - Starting DuckDB schema creation...
2025-08-06 19:32:07,209 - api.database.duckdb_schema - INFO - Starting DuckDB schema creation...
2025-08-06 19:32:07,209 - api.database.duckdb_schema - INFO - Starting DuckDB schema creation...


2025-08-06 19:32:07,231 - api.database.duckdb_schema - INFO - Creating documents table...
2025-08-06 19:32:07,231 - api.database.duckdb_schema - INFO - Creating documents table...
2025-08-06 19:32:07,231 - api.database.duckdb_schema - INFO - Creating documents table...
2025-08-06 19:32:07,328 - api.database.duckdb_schema - INFO - Documents table created successfully
2025-08-06 19:32:07,328 - api.database.duckdb_schema - INFO - Documents table created successfully
2025-08-06 19:32:07,328 - api.database.duckdb_schema - INFO - Documents table created successfully
2025-08-06 19:32:07,331 - api.database.duckdb_schema - INFO - Creating citations table...
2025-08-06 19:32:07,331 - api.database.duckdb_schema - INFO - Creating citations table...
2025-08-06 19:32:07,331 - api.database.duckdb_schema - INFO - Creating citations table...
2025-08-06 19:32:07,338 - api.database.duckdb_schema - INFO - Citations table created successfully
2025-08-06 19:32:07,338 - api.database.duckdb_schema - INFO - Ci

Using database: ../artifacts/mdc_challenge.db
Found 487 datasets!
First dataset: https://doi.org/10.11583/dtu.20555586


In [9]:
# # ✅ VERIFY: Check if cluster assignments are now working
# data_with_clusters = reduce_fixed.load_datasets_from_duckdb()
# print(f"Loaded {len(data_with_clusters)} datasets")

# # Check cluster distribution
# cluster_counts = {}
# for dataset in data_with_clusters:
#     cluster = dataset.cluster or "None"
#     cluster_counts[cluster] = cluster_counts.get(cluster, 0) + 1

# print(f"\nCluster assignments:")
# for cluster, count in sorted(cluster_counts.items()):
#     if cluster != "None":
#         print(f"  {cluster}: {count} datasets")

# print(f"\nFirst dataset:")
# print(f"  ID: {data_with_clusters[0].dataset_id}")
# print(f"  Cluster: {data_with_clusters[0].cluster}")
# print(f"  Type: {data_with_clusters[0].dataset_type}")

# datasets_with_clusters = len([d for d in data_with_clusters if d.cluster is not None])
# print(f"\n🎉 SUCCESS: {datasets_with_clusters}/{len(data_with_clusters)} datasets have cluster assignments!")


In [10]:
# data_fixed[0]

In [11]:
# data_fixed[-1].cluster

## UMAP

In [12]:
embeddings = reduce_fixed.load_embeddings_from_api()
len(embeddings)

2025-08-06 19:32:07,589 - src.helpers - INFO - Loading embeddings with URL: http://localhost:8000/load_embeddings
2025-08-06 19:32:07,589 - src.helpers - INFO - Loading embeddings with URL: http://localhost:8000/load_embeddings
2025-08-06 19:32:07,589 - src.helpers - INFO - Loading embeddings with URL: http://localhost:8000/load_embeddings
2025-08-06 19:32:07,593 - src.helpers - INFO - Using collection: dataset-aggregates-train
2025-08-06 19:32:07,593 - src.helpers - INFO - Using collection: dataset-aggregates-train
2025-08-06 19:32:07,593 - src.helpers - INFO - Using collection: dataset-aggregates-train
2025-08-06 19:32:08,141 - src.helpers - INFO - ✅ Successfully loaded 3 embeddings
2025-08-06 19:32:08,141 - src.helpers - INFO - ✅ Successfully loaded 3 embeddings
2025-08-06 19:32:08,141 - src.helpers - INFO - ✅ Successfully loaded 3 embeddings


487

In [13]:
DEFAULT_N_NEIGHBORS = 15
DEFAULT_MIN_DIST = 0.1
DEFAULT_N_COMPONENTS = 2
DEFAULT_RANDOM_SEED = 42

In [14]:
# Fix: Create fresh Reducer with active database connection and pre-loaded datasets
from src.umap import Reducer
fresh_reducer = Reducer(db_path="../artifacts/mdc_challenge.db")
fresh_reducer.datasets = data_fixed  # Pre-load datasets to avoid connection issues
print(f"Fresh reducer with {len(fresh_reducer.datasets)} pre-loaded datasets")

# Run UMAP reduction with fresh connection
res = fresh_reducer.run_umap_reduction(embeddings, n_neighbors=15, min_dist=0.1, n_components=2, random_seed=42)


/Users/taishajoseph/miniconda3/envs/mdc-challenge-2025/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-08-06 19:32:16,570 - src.helpers - INFO - Logging initialized for /Users/taishajoseph/Documents/Projects/MDC-Challenge-2025/logs/umap.log
2025-08-06 19:32:16,570 - src.helpers - INFO - Logging initialized for /Users/taishajoseph/Documents/Projects/MDC-Challenge-2025/logs/umap.log
2025-08-06 19:32:16,570 - src.helpers - INFO - Logging initialized for /Users/taishajoseph/Documents/Projects/MDC-Challenge-2025/logs/umap.log
2025-08-06 19:32:16,570 - src.helpers - INFO - Logging initialized for /Users/taishajoseph/Documents/Projects/MDC-Challenge-2025/logs/umap.log
2025-08-06 19:32:16,602 - api.database.duckdb_schema - INFO - Starting DuckDB schema creation...
2025-08-06 19:32:16,602 - api.database.duc

Fresh reducer with 487 pre-loaded datasets
Executing run_umap_reduction...
UMAP(n_jobs=1, random_state=42, verbose=True)
Wed Aug  6 19:32:16 2025 Construct fuzzy simplicial set
Wed Aug  6 19:32:17 2025 Finding Nearest Neighbors
Wed Aug  6 19:32:23 2025 Finished Nearest Neighbor Search
Wed Aug  6 19:32:26 2025 Construct embedding


Epochs completed:   5%| ▌          25/500 [00:01]

	completed  0  /  500 epochs


Epochs completed:  21%| ██         103/500 [00:01]

...

	completed  350  /  500 epochs
	completed  400  /  500 epochs


Epochs completed: 100%| ██████████ 500/500 [00:02]
2025-08-06 19:32:30,004 - src.helpers - INFO - UMAP embeddings shape: (487, 2)
2025-08-06 19:32:30,004 - src.helpers - INFO - UMAP embeddings shape: (487, 2)
2025-08-06 19:32:30,004 - src.helpers - INFO - UMAP embeddings shape: (487, 2)
2025-08-06 19:32:30,004 - src.helpers - INFO - UMAP embeddings shape: (487, 2)
2025-08-06 19:32:30,008 - src.helpers - INFO - Saving UMAP features for 487 datasets to DuckDB...
2025-08-06 19:32:30,008 - src.helpers - INFO - Saving UMAP features for 487 datasets to DuckDB...
2025-08-06 19:32:30,008 - src.helpers - INFO - Saving UMAP features for 487 datasets to DuckDB...
2025-08-06 19:32:30,008 - src.helpers - INFO - Saving UMAP features for 487 datasets to DuckDB...


	completed  450  /  500 epochs
Wed Aug  6 19:32:30 2025 Finished embedding


2025-08-06 19:32:30,361 - src.helpers - INFO - ✅ Upserted 1461 feature rows.
2025-08-06 19:32:30,361 - src.helpers - INFO - ✅ Upserted 1461 feature rows.
2025-08-06 19:32:30,361 - src.helpers - INFO - ✅ Upserted 1461 feature rows.
2025-08-06 19:32:30,361 - src.helpers - INFO - ✅ Upserted 1461 feature rows.
2025-08-06 19:32:30,404 - src.helpers - INFO - Database connection closed
2025-08-06 19:32:30,404 - src.helpers - INFO - Database connection closed
2025-08-06 19:32:30,404 - src.helpers - INFO - Database connection closed
2025-08-06 19:32:30,404 - src.helpers - INFO - Database connection closed
2025-08-06 19:32:30,407 - src.helpers - INFO - ✅ Successfully saved UMAP features for 487 datasets
2025-08-06 19:32:30,407 - src.helpers - INFO - ✅ Successfully saved UMAP features for 487 datasets
2025-08-06 19:32:30,407 - src.helpers - INFO - ✅ Successfully saved UMAP features for 487 datasets
2025-08-06 19:32:30,407 - src.helpers - INFO - ✅ Successfully saved UMAP features for 487 datasets


Function run_umap_reduction took 13.6329 seconds to complete.


In [15]:
fresh_reducer.create_cluster_visualization(embeddings)

2025-08-06 19:32:30,430 - src.helpers - INFO - 🎨 Creating cluster visualization...
2025-08-06 19:32:30,430 - src.helpers - INFO - 🎨 Creating cluster visualization...
2025-08-06 19:32:30,430 - src.helpers - INFO - 🎨 Creating cluster visualization...
2025-08-06 19:32:30,430 - src.helpers - INFO - 🎨 Creating cluster visualization...


Executing create_cluster_visualization...


2025-08-06 19:32:33,479 - src.helpers - INFO - ✅ Cluster visualization saved to: reports/dimensionality_reduction/cluster_visualization.png
2025-08-06 19:32:33,479 - src.helpers - INFO - ✅ Cluster visualization saved to: reports/dimensionality_reduction/cluster_visualization.png
2025-08-06 19:32:33,479 - src.helpers - INFO - ✅ Cluster visualization saved to: reports/dimensionality_reduction/cluster_visualization.png
2025-08-06 19:32:33,479 - src.helpers - INFO - ✅ Cluster visualization saved to: reports/dimensionality_reduction/cluster_visualization.png


Function create_cluster_visualization took 3.0534 seconds to complete.


True

In [16]:
fresh_reducer.create_ground_truth_visualization(embeddings)

2025-08-06 19:32:33,495 - src.helpers - INFO - 🎨 Creating ground truth visualization...
2025-08-06 19:32:33,495 - src.helpers - INFO - 🎨 Creating ground truth visualization...
2025-08-06 19:32:33,495 - src.helpers - INFO - 🎨 Creating ground truth visualization...
2025-08-06 19:32:33,495 - src.helpers - INFO - 🎨 Creating ground truth visualization...


Executing create_ground_truth_visualization...


2025-08-06 19:32:34,311 - src.helpers - INFO - ✅ Ground truth visualization saved to: reports/dimensionality_reduction/ground_truth_visualization.png
2025-08-06 19:32:34,311 - src.helpers - INFO - ✅ Ground truth visualization saved to: reports/dimensionality_reduction/ground_truth_visualization.png
2025-08-06 19:32:34,311 - src.helpers - INFO - ✅ Ground truth visualization saved to: reports/dimensionality_reduction/ground_truth_visualization.png
2025-08-06 19:32:34,311 - src.helpers - INFO - ✅ Ground truth visualization saved to: reports/dimensionality_reduction/ground_truth_visualization.png


Function create_ground_truth_visualization took 0.8205 seconds to complete.


True

In [17]:
import pandas as pd

res_df = pd.DataFrame(res)
res_df

,https://doi.org/10.11583/dtu.20555586,https://doi.org/10.5281/zenodo.8014150,ENSOARG00000003950,ENSOARG00000012128,ENSOARG00000012835,ENSOARG00000013782,ENSOARG00000013966,ENSOARG00000014129,IPR000264,IPR002172,...,https://doi.org/10.3334/cdiac/otg.ndp093_glodapv2,https://doi.org/10.6096/aeroclo.1754,https://doi.org/10.6096/aeroclo.1774,https://doi.org/10.6096/aeroclo.1777,https://doi.org/10.22033/esgf/cmip6.10843,https://doi.org/10.22033/esgf/cmip6.1621,https://doi.org/10.5066/f72j69rm,https://doi.org/10.15468/dl.a2a348,https://doi.org/10.15468/dl.k8zeq6,https://doi.org/10.6078/d1k419
0,-8.534961,13.646475,16.815632,16.847464,16.837645,16.81308,16.775257,16.787197,16.855883,16.794903,...,-9.549592,-9.545057,-8.820501,-8.808505,-8.729691,-8.762401,-9.656613,-8.821649,-8.845269,25.909603
1,4.728017,6.295977,3.153131,3.121337,3.181674,3.15744,3.195567,3.183709,3.115013,3.153053,...,4.783882,4.961493,5.552035,5.526095,5.033782,4.906552,3.494139,2.328143,2.384678,19.751556


In [18]:
# Step 1: Load full dataset using DuckDBHelper
from api.utils.duckdb_utils import DuckDBHelper

print("🔄 Loading full dataset from DuckDB...")

# Initialize DuckDB helper
db_path = "../artifacts/mdc_challenge.db"
db_helper = DuckDBHelper(db_path)

try:
    # Get full dataset dataframe with all engineered features
    df_full = db_helper.get_full_dataset_dataframe(
        dataset_ids=None,  # Include all datasets
        feature_filter=None,  # Include all features
        fill_na=True
    )
    
    print(f"✅ Dataset loaded successfully!")
    print(f"📊 Dataset shape: {df_full.shape}")
    print(f"📋 Columns: {list(df_full.columns)}")
    print(f"\n🔍 First few rows:")
    display(df_full.head())
    
except Exception as e:
    print(f"❌ Error loading dataset: {str(e)}")
    raise
db_helper.close()

2025-08-06 19:32:34,428 - api.database.duckdb_schema - INFO - Starting DuckDB schema creation...
2025-08-06 19:32:34,428 - api.database.duckdb_schema - INFO - Starting DuckDB schema creation...
2025-08-06 19:32:34,428 - api.database.duckdb_schema - INFO - Starting DuckDB schema creation...
2025-08-06 19:32:34,428 - api.database.duckdb_schema - INFO - Starting DuckDB schema creation...
2025-08-06 19:32:34,450 - api.database.duckdb_schema - INFO - Creating documents table...
2025-08-06 19:32:34,450 - api.database.duckdb_schema - INFO - Creating documents table...
2025-08-06 19:32:34,450 - api.database.duckdb_schema - INFO - Creating documents table...
2025-08-06 19:32:34,450 - api.database.duckdb_schema - INFO - Creating documents table...
2025-08-06 19:32:34,456 - api.database.duckdb_schema - INFO - Documents table created successfully
2025-08-06 19:32:34,456 - api.database.duckdb_schema - INFO - Documents table created successfully
2025-08-06 19:32:34,456 - api.database.duckdb_schema -

🔄 Loading full dataset from DuckDB...


2025-08-06 19:32:34,515 - api.database.duckdb_schema - INFO - Validating schema...
2025-08-06 19:32:34,643 - api.database.duckdb_schema - INFO - Table documents has 14 columns
2025-08-06 19:32:34,643 - api.database.duckdb_schema - INFO - Table documents has 14 columns
2025-08-06 19:32:34,643 - api.database.duckdb_schema - INFO - Table documents has 14 columns
2025-08-06 19:32:34,643 - api.database.duckdb_schema - INFO - Table documents has 14 columns
2025-08-06 19:32:34,649 - api.database.duckdb_schema - INFO - Table citations has 5 columns
2025-08-06 19:32:34,649 - api.database.duckdb_schema - INFO - Table citations has 5 columns
2025-08-06 19:32:34,649 - api.database.duckdb_schema - INFO - Table citations has 5 columns
2025-08-06 19:32:34,649 - api.database.duckdb_schema - INFO - Table citations has 5 columns
2025-08-06 19:32:34,656 - api.database.duckdb_schema - INFO - Table chunks has 6 columns
2025-08-06 19:32:34,656 - api.database.duckdb_schema - INFO - Table chunks has 6 columns

✅ Dataset loaded successfully!
📊 Dataset shape: (487, 65)
📋 Columns: ['dataset_id', 'document_id', 'total_tokens', 'avg_tokens_per_chunk', 'total_char_length', 'clean_text_length', 'cluster', 'dataset_type', 'text', 'created_at', 'updated_at', 'LEIDEN_1', 'LEIDEN_cluster_0', 'LEIDEN_cluster_1', 'LEIDEN_cluster_10', 'LEIDEN_cluster_11', 'LEIDEN_cluster_12', 'LEIDEN_cluster_13', 'LEIDEN_cluster_14', 'LEIDEN_cluster_15', 'LEIDEN_cluster_16', 'LEIDEN_cluster_17', 'LEIDEN_cluster_18', 'LEIDEN_cluster_19', 'LEIDEN_cluster_2', 'LEIDEN_cluster_20', 'LEIDEN_cluster_21', 'LEIDEN_cluster_22', 'LEIDEN_cluster_23', 'LEIDEN_cluster_24', 'LEIDEN_cluster_25', 'LEIDEN_cluster_26', 'LEIDEN_cluster_27', 'LEIDEN_cluster_28', 'LEIDEN_cluster_29', 'LEIDEN_cluster_3', 'LEIDEN_cluster_30', 'LEIDEN_cluster_31', 'LEIDEN_cluster_32', 'LEIDEN_cluster_33', 'LEIDEN_cluster_34', 'LEIDEN_cluster_35', 'LEIDEN_cluster_36', 'LEIDEN_cluster_37', 'LEIDEN_cluster_38', 'LEIDEN_cluster_39', 'LEIDEN_cluster_4', 'LEIDEN_cluste

,dataset_id,document_id,total_tokens,avg_tokens_per_chunk,total_char_length,clean_text_length,cluster,dataset_type,text,created_at,...,LEIDEN_cluster_48,LEIDEN_cluster_49,LEIDEN_cluster_5,LEIDEN_cluster_50,LEIDEN_cluster_6,LEIDEN_cluster_7,LEIDEN_cluster_8,LEIDEN_cluster_9,UMAP_1,UMAP_2
0,https://doi.org/10.11583/dtu.20555586,10.1016_j.fuel.2022.125768,1781,593.666687,6288,6263,cluster_30,PRIMARY,"(SOI), total λ, and ammonia energy. Increased ...",2025-08-05 16:12:25.321621,...,-0.057139,-0.023878,0.041667,-0.007818,0.041906,-0.002887,0.053897,-0.047923,-8.534961,4.728017
1,https://doi.org/10.5281/zenodo.8014150,10.1016_j.jobe.2023.107105,1235,411.666656,6118,6092,cluster_27,PRIMARY,cleaning and linking. Several heuristic method...,2025-08-05 16:12:25.321639,...,-0.073679,-0.006766,0.045149,-0.000470,0.081533,-0.039715,0.029550,-0.009841,13.646475,6.295977
2,ENSOARG00000003950,10.1038_s41598-020-59839-x,2093,697.666687,7321,7303,cluster_18,SECONDARY,"the glucocorticoid axis, the MC5R gene regulat...",2025-08-05 16:12:25.321646,...,0.012372,0.010772,-0.026317,0.004461,-0.018842,-0.013471,0.010239,0.035581,16.815632,3.153131
3,ENSOARG00000012128,10.1038_s41598-020-59839-x,2093,697.666687,7321,7315,cluster_18,SECONDARY,"the glucocorticoid axis, the MC5R gene regulat...",2025-08-05 16:12:25.321652,...,0.012372,0.010772,-0.026317,0.004461,-0.018842,-0.013471,0.010239,0.035581,16.847464,3.121337
4,ENSOARG00000012835,10.1038_s41598-020-59839-x,2110,703.333313,7189,7141,cluster_18,SECONDARY,sum of their lengths (in bp) and the sum of th...,2025-08-05 16:12:25.321657,...,0.024234,0.029056,-0.022071,0.024005,-0.011032,-0.033191,0.025962,0.025353,16.837645,3.181674


2025-08-06 19:32:34,827 - src.helpers - INFO - Database connection closed
2025-08-06 19:32:34,827 - src.helpers - INFO - Database connection closed
2025-08-06 19:32:34,827 - src.helpers - INFO - Database connection closed
2025-08-06 19:32:34,827 - src.helpers - INFO - Database connection closed


## PCA

In [ ]:
# Fix: Create fresh Reducer with active database connection and pre-loaded datasets
from src.dimensionality_reduction import Reducer
fresh_reducer = Reducer(db_path="../artifacts/mdc_challenge.db")
fresh_reducer.datasets = data_fixed  # Pre-load datasets to avoid connection issues
print(f"Fresh reducer with {len(fresh_reducer.datasets)} pre-loaded datasets")

# Run UMAP reduction with fresh connection
res = fresh_reducer.run_umap_reduction(embeddings, n_neighbors=15, min_dist=0.1, n_components=2, random_seed=42)


In [20]:
# Load feature clusters
feature_cluster_map = fresh_reducer.load_feature_clusters(os.path.join(project_root, "reports/clustering/feature_clusters.json"))
len(feature_cluster_map)

2025-08-05 21:05:29,270 - src.helpers - INFO - Loading feature clusters from /Users/taishajoseph/Documents/Projects/MDC-Challenge-2025/reports/clustering/feature_clusters.json...
2025-08-05 21:05:29,270 - src.helpers - INFO - Loading feature clusters from /Users/taishajoseph/Documents/Projects/MDC-Challenge-2025/reports/clustering/feature_clusters.json...
2025-08-05 21:05:29,270 - src.helpers - INFO - Loading feature clusters from /Users/taishajoseph/Documents/Projects/MDC-Challenge-2025/reports/clustering/feature_clusters.json...
2025-08-05 21:05:29,275 - src.helpers - INFO - ✅ Successfully loaded 384 feature cluster mappings
2025-08-05 21:05:29,275 - src.helpers - INFO - ✅ Successfully loaded 384 feature cluster mappings
2025-08-05 21:05:29,275 - src.helpers - INFO - ✅ Successfully loaded 384 feature cluster mappings


384

In [22]:
import numpy as np

# Convert embeddings dict to matrix format
dataset_ids = list(embeddings.keys())
dataset_embeddings = np.array([embeddings[ds_id] for ds_id in dataset_ids])

In [23]:
res_pca = fresh_reducer.run_per_cluster_pca(dataset_embeddings,feature_cluster_map=feature_cluster_map,dataset_ids=dataset_ids, random_seed = 42)

2025-08-05 21:05:55,174 - src.helpers - INFO - 🔄 Running feature-cluster PCA...
2025-08-05 21:05:55,174 - src.helpers - INFO - 🔄 Running feature-cluster PCA...
2025-08-05 21:05:55,174 - src.helpers - INFO - 🔄 Running feature-cluster PCA...
2025-08-05 21:05:55,177 - src.helpers - INFO - Processing 51 feature clusters
2025-08-05 21:05:55,177 - src.helpers - INFO - Processing 51 feature clusters
2025-08-05 21:05:55,177 - src.helpers - INFO - Processing 51 feature clusters
2025-08-05 21:05:55,186 - src.helpers - INFO - cluster_6: PC1 var ratio = 0.3765
2025-08-05 21:05:55,190 - src.helpers - INFO - cluster_30: PC1 var ratio = 0.4035
2025-08-05 21:05:55,186 - src.helpers - INFO - cluster_6: PC1 var ratio = 0.3765
2025-08-05 21:05:55,186 - src.helpers - INFO - cluster_6: PC1 var ratio = 0.3765
2025-08-05 21:05:55,191 - src.helpers - INFO - cluster_10: PC1 var ratio = 0.3343
2025-08-05 21:05:55,190 - src.helpers - INFO - cluster_30: PC1 var ratio = 0.4035
2025-08-05 21:05:55,191 - src.helpers

Executing run_per_cluster_pca...


2025-08-05 21:05:55,351 - src.helpers - INFO - cluster_7: PC1 var ratio = 0.4359
2025-08-05 21:05:55,360 - src.helpers - INFO - cluster_23: PC1 var ratio = 0.2732
2025-08-05 21:05:55,353 - src.helpers - INFO - cluster_44: PC1 var ratio = 0.3341
2025-08-05 21:05:55,364 - src.helpers - INFO - cluster_15: PC1 var ratio = 0.2862
2025-08-05 21:05:55,353 - src.helpers - INFO - cluster_44: PC1 var ratio = 0.3341
2025-08-05 21:05:55,360 - src.helpers - INFO - cluster_23: PC1 var ratio = 0.2732
2025-08-05 21:05:55,360 - src.helpers - INFO - cluster_23: PC1 var ratio = 0.2732
2025-08-05 21:05:55,368 - src.helpers - INFO - cluster_9: PC1 var ratio = 0.3156
2025-08-05 21:05:55,364 - src.helpers - INFO - cluster_15: PC1 var ratio = 0.2862
2025-08-05 21:05:55,364 - src.helpers - INFO - cluster_15: PC1 var ratio = 0.2862
2025-08-05 21:05:55,372 - src.helpers - INFO - cluster_20: PC1 var ratio = 0.3408
2025-08-05 21:05:55,368 - src.helpers - INFO - cluster_9: PC1 var ratio = 0.3156
2025-08-05 21:05:55

Function run_per_cluster_pca took 1.7332 seconds to complete.


In [ ]:
from api.utils.duckdb_utils import DuckDBHelper
correct_db_path = "../artifacts/mdc_challenge.db"
db_helper = DuckDBHelper(correct_db_path)

db_helper.close()

2025-08-05 21:07:11,524 - api.database.duckdb_schema - INFO - Starting DuckDB schema creation...
2025-08-05 21:07:11,524 - api.database.duckdb_schema - INFO - Starting DuckDB schema creation...
2025-08-05 21:07:11,524 - api.database.duckdb_schema - INFO - Starting DuckDB schema creation...
2025-08-05 21:07:11,551 - api.database.duckdb_schema - INFO - Creating documents table...
2025-08-05 21:07:11,551 - api.database.duckdb_schema - INFO - Creating documents table...
2025-08-05 21:07:11,551 - api.database.duckdb_schema - INFO - Creating documents table...
2025-08-05 21:07:11,555 - api.database.duckdb_schema - INFO - Documents table created successfully
2025-08-05 21:07:11,555 - api.database.duckdb_schema - INFO - Documents table created successfully
2025-08-05 21:07:11,555 - api.database.duckdb_schema - INFO - Documents table created successfully
2025-08-05 21:07:11,558 - api.database.duckdb_schema - INFO - Creating citations table...
2025-08-05 21:07:11,558 - api.database.duckdb_schema

: 